__Author:__ Bram Van de Sande
   
__Date:__ 9 FEB 2018

__Outline:__ Characterize the different cells in a single-cell transcriptomics experiment by the enrichment of the regulomes. Enrichment of a regulome is measures as AUC of the recovery curve of the genes that define this regulome.

In [1]:
import pickle
import os
import pandas as pd
from collections import defaultdict
from pyscenic.aucell import create_rankings, enrichment
from pyscenic.regulome import df2regulomes
import seaborn

In [2]:
RESOURCES_FOLDER="/Users/bramvandesande/Projects/lcb/resources"
DATA_FOLDER="/Users/bramvandesande/Projects/lcb/tmp"

Load and rank expression profiles from single-cell experiment.

In [3]:
ex_mtx = pd.read_csv(os.path.join(RESOURCES_FOLDER, 'GSE60361_C1-3005-Expression.txt'), sep='\t', header=0, index_col=0)

In [4]:
rnk_mtx = create_rankings(ex_mtx)

In [5]:
rnk_mtx.T.shape

(3005, 19972)

Load regulomes discovered in previous phase.

In [6]:
df = pd.read_csv(os.path.join(RESOURCES_FOLDER, "regulomes_zeisel_2015.csv"),
                 index_col=[0,1], header=[0,1], skipinitialspace=True)

In [7]:
df[('Enrichment', 'TargetGenes')] = df[('Enrichment', 'TargetGenes')].apply(lambda e: eval(e))

In [8]:
df.head()

Enrichment            \
                                            AUC       NES   
TF     MotifID                                              
Acaa1b transfac_pro__M01543            0.076742  4.846509   
       transfac_pro__M04671            0.076380  4.821018   
Ar     homer__CCAGGAACAG_AR-halfsite   0.050721  7.485099   
Arid3a transfac_pro__M04727            0.033484  3.115786   
       cisbp__M4455                    0.035068  3.291999   

                                                            \
                                     MotifSimilarityQvalue   
TF     MotifID                                               
Acaa1b transfac_pro__M01543                       0.000000   
       transfac_pro__M04671                       0.000000   
Ar     homer__CCAGGAACAG_AR-halfsite              0.000000   
Arid3a transfac_pro__M04727                       0.000255   
       cisbp__M4455                               0.000009   

                                                          \
                                     OrthologousIdentity   
TF     MotifID                                             
Acaa1b transfac_pro__M01543                     0.517986   
       transfac_pro__M04671                     0.517986   
Ar     homer__CCAGGAACAG_AR-halfsite            0.877642   
Arid3a transfac_pro__M04727                     0.798669   
       cisbp__M4455                             0.798669   

                                                                                         \
                                                                             Annotation   
TF     MotifID                                                                            
Acaa1b transfac_pro__M01543           motif similar to transfac_pro__M04671 ('F$POT1...   
       transfac_pro__M04671           motif is annotated for orthologous gene YIL160...   
Ar     homer__CCAGGAACAG_AR-halfsite  gene is orthologous to ENSG00000169083 in H. s...   
Arid3a transfac_pro__M04727           gene is orthologous to ENSG00000116017 in H. s...   
       cisbp__M4455                   gene is orthologous to ENSG00000116017 in H. s...   

                                                                     \
                                                            Context   
TF     MotifID                                                        
Acaa1b transfac_pro__M01543           frozenset({'top50', '500bp'})   
       transfac_pro__M04671           frozenset({'top50', '500bp'})   
Ar     homer__CCAGGAACAG_AR-halfsite  frozenset({'top50', '500bp'})   
Arid3a transfac_pro__M04727           frozenset({'top50', '500bp'})   
       cisbp__M4455                   frozenset({'top50', '500bp'})   

                                                                                         
                                                                            TargetGenes  
TF     MotifID                                                                           
Acaa1b transfac_pro__M01543           [(Col7a1, 105), (Lmna, 107), (Crh, 109), (Htr1...  
       transfac_pro__M04671           [(Col7a1, 101), (Lmna, 106), (Crh, 109), (Htr1...  
Ar     homer__CCAGGAACAG_AR-halfsite          [(Bmpr1b, 17), (Kcns1, 45), (Ptpru, 119)]  
Arid3a transfac_pro__M04727                                                 [(Sdk2, 6)]  
       cisbp__M4455                                     [(Hbegf, 30), (Tnfaip8l2, 102)]

In [9]:
regulomes = df2regulomes(df, "MGI")

In [10]:
len(regulomes)

423

In [11]:
regulomes[0]

Regulome(name='Acaa1b', nomenclature='MGI', gene2weights=<frozendict {'Col7a1': 1.0, 'Lmna': 1.0, 'Crh': 1.0, 'Htr1f': 1.0, 'Rilpl2': 1.0, 'Phf21b': 1.0, 'Poli': 1.0, 'Mtfr1': 1.0}>, transcription_factor='Acaa1b', context=frozenset(), score=2.510423682097521)

Calculate enrichment as AUC (NES is not valid because AUC are not normally distributed) of regulomes in cells.

Best to calculate the rankings for a subset of the regulomes, i.e. per database and regulome definition.

In [12]:
auc_heatmap = pd.concat([enrichment(rnk_mtx.T, regulome) for regulome in regulomes]).unstack('Regulome')

In [13]:
auc_heatmap

AUC                                                    \
Regulome          Acaa1b       Ahr        Ar    Arid3a    Arid5b     Arnt2   
Cell                                                                         
1772058148_A01  0.000000  0.036427  0.000000  0.000000  0.000000  0.037643   
1772058148_A03  0.000000  0.037728  0.014948  0.000000  0.097264  0.043805   
1772058148_A04  0.323073  0.082962  0.164031  0.000000  0.013847  0.085080   
1772058148_A05  0.000000  0.054247  0.000000  0.053171  0.009927  0.025727   
1772058148_A06  0.000000  0.061429  0.000000  0.000000  0.014181  0.054170   
1772058148_A07  0.000000  0.053367  0.000000  0.000000  0.076743  0.031977   
1772058148_A09  0.000000  0.104423  0.021088  0.111288  0.154571  0.064974   
1772058148_A10  0.000000  0.036157  0.000000  0.000000  0.067568  0.046995   
1772058148_A11  0.122998  0.051244  0.021154  0.077077  0.052135  0.047437   
1772058148_A12  0.149775  0.066609  0.000000  0.000000  0.062563  0.051021   
1772058148_B01  0.000000  0.035479  0.059593  0.043750  0.000000  0.039513   
1772058148_B02  0.119244  0.013134  0.000000  0.000000  0.000000  0.028019   
1772058148_B03  0.106481  0.024482  0.060527  0.051875  0.000000  0.021174   
1772058148_B06  0.000000  0.036866  0.000000  0.000000  0.081999  0.031701   
1772058148_B08  0.107608  0.064248  0.102569  0.000000  0.040707  0.134813   
1772058148_B11  0.114865  0.033778  0.000000  0.000000  0.000000  0.033158   
1772058148_B12  0.119119  0.039461  0.044578  0.038215  0.062896  0.033334   
1772058148_C01  0.000000  0.028567  0.000000  0.098451  0.047047  0.016170   
1772058148_C02  0.000000  0.065334  0.000000  0.030913  0.104271  0.057828   
1772058148_C03  0.332833  0.037567  0.000000  0.092269  0.006340  0.054951   
1772058148_C04  0.000000  0.015235  0.000000  0.031325  0.000000  0.016429   
1772058148_C05  0.000000  0.033760  0.174441  0.043926  0.056223  0.025949   
1772058148_C07  0.227728  0.024141  0.059059  0.000000  0.000000  0.018758   
1772058148_C09  0.000000  0.038343  0.161695  0.050816  0.082749  0.031375   
1772058148_D02  0.000000  0.024854  0.000000  0.055821  0.000000  0.021031   
1772058148_D03  0.029404  0.054261  0.000000  0.111347  0.000000  0.030489   
1772058148_D04  0.000000  0.064410  0.000000  0.000000  0.050050  0.030715   
1772058148_D07  0.000000  0.053767  0.000000  0.000000  0.067651  0.060359   
1772058148_D08  0.625876  0.580239  0.600400  0.704999  0.647564  0.390430   
1772058148_D10  0.000000  0.065842  0.000000  0.053524  0.000000  0.028276   
...                  ...       ...       ...       ...       ...       ...   
1772071044_H01  0.220971  0.201385  0.287955  0.200730  0.101351  0.225238   
1772071044_H04  0.237237  0.218676  0.408141  0.214568  0.177678  0.229555   
1772071044_H06  0.196321  0.128038  0.177578  0.000000  0.176927  0.232281   
1772071044_H10  0.548549  0.596004  0.543010  0.679680  0.684268  0.578124   
1772071044_H11  0.000000  0.048968  0.173173  0.043809  0.000000  0.042228   
1772071044_H12  0.209334  0.124438  0.089423  0.000000  0.130130  0.173735   
1772071045_A01  0.688438  0.633005  0.680614  0.805924  0.676593  0.463740   
1772071045_A04  0.094595  0.140356  0.268869  0.000000  0.183016  0.230950   
1772071045_A07  0.121997  0.242889  0.432499  0.038980  0.342759  0.470067   
1772071045_A10  0.644645  0.525104  0.511778  0.752635  0.622873  0.414670   
1772071045_B06  0.066817  0.180907  0.321522  0.202732  0.276777  0.322722   
1772071045_C02  0.000000  0.036391  0.143343  0.000000  0.000000  0.020410   
1772071045_C11  0.667417  0.623592  0.577644  0.708473  0.695696  0.548798   
1772071045_D02  0.000000  0.049372  0.052920  0.000000  0.000000  0.049613   
1772071045_D04  0.226852  0.110532  0.171438  0.000000  0.156657  0.243013   
1772071045_D05  0.110485  0.046984  0.130998  0.089913  0.000000  0.066707   
1772071045_D06  0.686562  0.648047  0.641108  0.770830  0.646647  0.472274   
1772071045_E10  0.000000  0.120775  0.10343

In [ ]:
ax = seaborn.heatmap(auc_heatmap)